## Trainning A Classifier

### 1. Loading and normalizing CIFAR10

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms


In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)


testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)


Files already downloaded and verified
Files already downloaded and verified


In [3]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

### 2. Define a Convolution Neural Network

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class Network(nn.Module):
    
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
    
network = Network()

### 3. Define a Loss function and optimizer

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(network.parameters(), lr=0.001, momentum=0.9)

### 4. Train the network

In [6]:
for epoch in range(2): # loop over the dataset multiple times
    running_loss = 0.0
    
    for i, data in enumerate(trainloader):
        # get the inputs
        inputs, labels = data
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = network(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print results
        running_loss += loss.item()
        
        if i % 2000 == 1999:
            print('[{}, {}] loss: {:.3f}'.format(epoch+1, i+1, running_loss/2000))
            running_loss = 0.0
            
print('Finished Training')
        

[1, 2000] loss: 2.196
[1, 4000] loss: 1.867
[1, 6000] loss: 1.662
[1, 8000] loss: 1.554
[1, 10000] loss: 1.485
[1, 12000] loss: 1.444
[2, 2000] loss: 1.375
[2, 4000] loss: 1.358
[2, 6000] loss: 1.330
[2, 8000] loss: 1.316
[2, 10000] loss: 1.282
[2, 12000] loss: 1.288
Finished Training


### 5. Test the network on the test data


In [7]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = network(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print("Accuracy of the network in the 10000 test images: {} %".format(100 * correct / total))

Accuracy of the network in the 10000 test images: 54.92 %


In [8]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = network(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print("Accuracy of {} : {} %".format(classes[i], 100 * class_correct[i] / class_total[i]))


Accuracy of plane : 64.3 %
Accuracy of car : 83.8 %
Accuracy of bird : 20.3 %
Accuracy of cat : 29.6 %
Accuracy of deer : 50.6 %
Accuracy of dog : 57.9 %
Accuracy of frog : 61.5 %
Accuracy of horse : 68.0 %
Accuracy of ship : 64.3 %
Accuracy of truck : 48.9 %
